In [1]:
%env HCA_ANALYTICS_REPORTING_CLIENT_SECRET_PATH=../../do_not_commit_ga4_credentials.json

env: HCA_ANALYTICS_REPORTING_CLIENT_SECRET_PATH=../../do_not_commit_ga4_credentials.json


In [2]:
import analytics.api as ga
import analytics.sheets_api as sheets
import analytics.sheets_elements as elements
import pandas as pd
import gspread
from constants import *

In [3]:
ga_authentication, drive_authentication, sheets_authentication = ga.authenticate(
    SECRET_NAME,
    ga.ga4_service_params,
    ga.drive_service_params,
    ga.sheets_service_params,
    port=OAUTH_PORT
)

REPORT_DATES = elements.get_bounds_for_month_and_prev(CURRENT_MONTH)
START_DATE_CURRENT = REPORT_DATES["start_current"]
END_DATE_CURRENT = REPORT_DATES["end_current"]
START_DATE_PRIOR = REPORT_DATES["start_previous"]
END_DATE_PRIOR = REPORT_DATES["end_previous"]

date_string = f"{START_DATE_CURRENT} - {END_DATE_CURRENT}"

default_params = {
    "service_system": ga_authentication,
    "start_date": START_DATE_CURRENT,
    "end_date": END_DATE_CURRENT,
    "property": HCA_PORTAL_ID,
}

hca_portal_params = {
    **default_params,
    "base_dimension_filter": HCA_PORTAL_ONLY_FILTER,
}

hca_combined_all_time_params = {
    **default_params,
    "start_date": GA4_START,
}

hca_portal_all_time_params = {
    **default_params,
    "start_date": GA4_START_SEPARATE,
    "base_dimension_filter": HCA_PORTAL_ONLY_FILTER,

}

hca_explorer_all_time_params = {
    **default_params,
    "start_date": GA4_START_SEPARATE,
    "base_dimension_filter": HCA_EXPLORER_ONLY_FILTER,
}

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=274560362763-p5netdrssq6r02lcfan6s157m6d65rqe.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8082%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fspreadsheets+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fanalytics.readonly+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&state=S939XE4HSQY9zVyap3fxpYE1Mr6R4D&access_type=offline


In [4]:
df_monthly_summary_portal = elements.get_page_views_over_time_df(hca_portal_all_time_params)
df_monthly_summary_explorer = elements.get_page_views_over_time_df(hca_explorer_all_time_params)
df_monthly_summary_combined = elements.get_page_views_over_time_df(hca_combined_all_time_params, additional_data_path=HISTORIC_UA_DATA_PATH, additional_data_behavior=elements.ADDITIONAL_DATA_BEHAVIOR.ADD)
df_outbound = elements.get_outbound_links_change(hca_portal_params, START_DATE_CURRENT, END_DATE_CURRENT, START_DATE_PRIOR, END_DATE_PRIOR)
df_pageviews = elements.get_page_views_change(hca_portal_params, START_DATE_CURRENT, END_DATE_CURRENT, START_DATE_PRIOR, END_DATE_PRIOR)

In [5]:
dict_spreadsheet = {
    "Combined Summary": df_monthly_summary_combined,
    "Portal Summary": df_monthly_summary_portal,
    "Explorer Summary": df_monthly_summary_explorer,
    "Page Views (Portal)": df_pageviews,
    "Outbound Links (Portal)": df_outbound,
}

summary_worksheet_formatting = {
    "extra_columns": 1,
    "extra_columns_width": 2000,
}

sheet = sheets.create_sheet_in_folder(
    drive_authentication,
    SHEET_NAME,
    PARENT_FOLDER_NAME,
    override_behavior=sheets.FILE_OVERRIDE_BEHAVIORS.OVERRIDE_IF_IN_SAME_PLACE
)

sheets.fill_spreadsheet_with_df_dict(
    sheet,
    dict_spreadsheet,
    sheets.FILE_OVERRIDE_BEHAVIORS.OVERRIDE_IF_IN_SAME_PLACE,
    column_formatting_options={
        "Combined Summary": elements.PAGE_VIEWS_OVER_TIME_FORMATTING,
        "Portal Summary": elements.PAGE_VIEWS_OVER_TIME_FORMATTING,
        "Explorer Summary": elements.PAGE_VIEWS_OVER_TIME_FORMATTING,
        "Page Views (Portal)": elements.PAGE_VIEWS_CHANGE_FORMATTING,
        "Outbound Links (Portal)": elements.OUTBOUND_LINKS_CHANGE_FORMATTING
    },
    sheet_formatting_options={
        "Portal Summary": summary_worksheet_formatting,
        "Explorer Summary": summary_worksheet_formatting,
        "Combined Summary": summary_worksheet_formatting
    }
)
for df, worksheet_name in zip(
    (df_monthly_summary_portal, df_monthly_summary_explorer, df_monthly_summary_combined), 
    ("Portal Summary", "Explorer Summary", "Combined Summary")
):
    worksheet = sheet.worksheet(worksheet_name)
    date_range = sheets.WorksheetRange(
        worksheet, 
        gspread.cell.Cell(1, 1), 
        gspread.cell.Cell(df.index.size + 1, 2)
    )
    users_range = sheets.WorksheetRange(
        worksheet, 
        gspread.cell.Cell(1, 2), 
        gspread.cell.Cell(df.index.size + 1, 3)
    )
    pageviews_range = sheets.WorksheetRange(
        worksheet, 
        gspread.cell.Cell(1, 3), 
        gspread.cell.Cell(df.index.size + 1, 4)
    )
    sheets.add_chart_to_sheet(
        sheets_authentication,
        sheet,
        worksheet,
        sheets.CHART_TYPES.LINE,
        date_range,
        [users_range, pageviews_range],
        chart_position=gspread.cell.Cell(1, 6),
        chart_position_offset_x=75,
        chart_position_offset_y=25,
        title="Pageviews and Users Over Time"
    )